In [ ]:
import requests
from bs4 import BeautifulSoup
import pymysql

word = '빅데이터'
books = []
conn = pymysql.connect(user = 'maria', password = '1234',
                       host = 'localhost', port = 3306,
                       database = 'python_db')
cursor = conn.cursor()

for page in range(1, 6):
    url = f'https://www.yes24.com/Product/Search?domain=ALL&query={word}&page={page}'

    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        lis = soup.select('#yesSchList > li')
        for li in lis:
            try:
                bcode = li.get('data-goods-no')
                title = li.select_one('.gd_name').text.strip()
                author = li.select_one('.authPub.info_auth > a').text.strip()
                publisher = li.select_one('.authPub.info_pub > a').text.strip()
                sql = f'''select bscode from book_store where name = '{publisher}' '''
                cursor.execute(sql)
                pub_code = cursor.fetchone()
                if not pub_code:
                    pub_code = [0]

                year_of_publication = li.select_one('.authPub.info_date').text.strip()
                price = li.select_one('.info_price em.yes_b').text.replace(',', '')
                books.append([bcode, title, author, year_of_publication, pub_code[0], price])
            except:
                continue

    else:
        print(response.status_code)
        
sql = '''insert into book_list values (%s, %s, %s, %s, %s, %s)'''

for book in books:
    try:
        cursor.execute(sql, book)
    except:
        print('duplicate')
        continue
conn.commit()
conn.close()